<a href="https://colab.research.google.com/github/EdiNel0407/us-ie-big-data-technologies/blob/main/day3/day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget -q -O rows.csv "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv"
import pandas as pd
pd.read_csv("rows.csv").head()


/tmp/ipython-input-1458916647.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("rows.csv").head()


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,09/21/2022,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [6]:
!wget -q -O users_v.csv https://storage.googleapis.com/bdt-beam/users_v.csv
!ls -lh users_v.csv && head -n 5 users_v.csv



-rw-r--r-- 1 root root 141K Nov  4  2021 users_v.csv
user_id,name,gender,age,address,date_joined
1,Anthony Wolf,male,73,New Rachelburgh-VA-49583,2019/03/13
2,James Armstrong,male,56,North Jillianfort-UT-86454,2020/11/06
3,Cody Shaw,male,75,North Anne-SC-53799,2004/05/29
4,Sierra Hamilton,female,76,New Angelafurt-ME-46190,2005/08/26


In [7]:
!wget -q -O rows.csv "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv"
!ls -lh rows.csv && head -n 3 rows.csv


-rw-r--r-- 1 root root 444M Sep 15 22:52 rows.csv
CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
09/11/2021,2:39,,,,,,WHITESTONE EXPRESSWAY,20 AVENUE,,2,0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,,,,4455765,Sedan,Sedan,,,
03/26/2022,11:45,,,,,,QUEENSBORO BRIDGE UPPER,,,1,0,0,0,0,0,1,0,Pavement Slippery,,,,,4513547,Sedan,,,,


In [8]:
pdf = pd.read_csv("rows.csv")
nyc_small = pdf.sample(frac=0.2, replace=False, random_state=1)[[
    "CRASH DATE",
    "CONTRIBUTING FACTOR VEHICLE 1",
    "BOROUGH",
    "VEHICLE TYPE CODE 1",
    "NUMBER OF PERSONS INJURED"
]]

sdf = spark.createDataFrame(nyc_small.fillna("").astype(str)).cache()
_ = sdf.count()
sdf.printSchema()


/tmp/ipython-input-2260987281.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pdf = pd.read_csv("rows.csv")


root
 |-- CRASH DATE: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- VEHICLE TYPE CODE 1: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)



In [11]:
(
  sdf.withColumn("CRASH_DATE", F.to_date(F.col("CRASH DATE"), "MM/dd/yyyy"))
     .groupBy("CRASH_DATE").count()
     .orderBy(F.desc("count"))
     .show(5, truncate=False)
)


+----------+-----+
|CRASH_DATE|count|
+----------+-----+
|2014-01-21|250  |
|2018-11-15|232  |
|2017-05-19|199  |
|2014-02-03|193  |
|2015-01-18|192  |
+----------+-----+
only showing top 5 rows



In [12]:
sdf.filter(F.col("BOROUGH") != "") \
   .groupBy("BOROUGH").count() \
   .orderBy(F.desc("count")).show(5, truncate=False)


+-------------+-----+
|BOROUGH      |count|
+-------------+-----+
|BROOKLYN     |97946|
|QUEENS       |82045|
|MANHATTAN    |67597|
|BRONX        |45203|
|STATEN ISLAND|12921|
+-------------+-----+



In [13]:
sdf.filter(
    (F.col("BOROUGH") == "QUEENS") &
    (F.col("CONTRIBUTING FACTOR VEHICLE 1") != "") &
    (F.lower(F.col("CONTRIBUTING FACTOR VEHICLE 1")) != "unspecified")
).groupBy("CONTRIBUTING FACTOR VEHICLE 1") \
 .agg(F.count("*").alias("freq")) \
 .orderBy(F.desc("freq")).show(5, truncate=False)


+------------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1 |freq |
+------------------------------+-----+
|Driver Inattention/Distraction|17724|
|Failure to Yield Right-of-Way |7174 |
|Backing Unsafely              |4004 |
|Following Too Closely         |3012 |
|Passing or Lane Usage Improper|2236 |
+------------------------------+-----+
only showing top 5 rows



In [14]:
sdf.withColumn("NUM_INJ", F.col("NUMBER OF PERSONS INJURED").cast("int")) \
   .groupBy("VEHICLE TYPE CODE 1","BOROUGH") \
   .agg(F.avg("NUM_INJ").alias("avg_injuries")) \
   .orderBy(F.desc("avg_injuries")).show(3, truncate=False)


+-------------------+--------+------------+
|VEHICLE TYPE CODE 1|BOROUGH |avg_injuries|
+-------------------+--------+------------+
|bobct              |BROOKLYN|5.0         |
|FDNY FIRE          |        |4.0         |
|suv                |QUEENS  |4.0         |
+-------------------+--------+------------+
only showing top 3 rows

